# Exploring neural networks

## Problem 2

Given a dataset where each sample contains all 0s and randomly a number between 0 and 100 somewhere to replace the 0s.
Create a neural network that can distinguish between record with no replaced number or a number below 50 and those that contain 0s and a number above 50.

## Problem 3

Given a datase where each sample contains random numbers between 0-10, create a neural network that can distinguish between record whose sum is greater than 20 and those that are less than 20.

In [2]:
import numpy as np
import random

In [35]:
def generate_dataset(size = 1000):

    inp = []
    labels =[]

    for i in range(size):
        inp.append(np.zeros(11))
        labels.append(0)
        rnd_choice = random.choice(range(101))
        inp[i][random.choice(range(10))] = rnd_choice
        labels[i] = 1 if rnd_choice > 50 else 0
   
    return (np.array(inp), labels)

ds = generate_dataset(100000)


In [107]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(11, 6, 3)
#         self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(6 * 3 * 3, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
#         x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(11, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=54, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [108]:
round(ds[0].shape[0] * 0.8)

80000

In [109]:
def torch_train_test_data(ds, split_percent=0.8):
    
    split = round(ds[0].shape[0] * 0.8)
    
    x_train = torch.tensor(ds[0][0:split], dtype=torch.float32)
    y_train = torch.tensor(ds[1][0:split], dtype=torch.long)
    
    train_data = data.TensorDataset(x_train, y_train)

    x_test = torch.tensor(ds[0][split:], dtype=torch.float32)
    y_test = torch.tensor(ds[1][split:], dtype=torch.long)

    test_data = data.TensorDataset(x_test, y_test)
       
    return (x_train, y_train, x_test, y_test, train_data, test_data)


x_train, y_train, x_test, y_test, train_data, test_data = torch_train_test_data(ds, 0.8)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=11,
                                         shuffle=False, num_workers=0) 
    
test_loader = torch.utils.data.DataLoader(test_data, batch_size=11,
                                         shuffle=False, num_workers=0)


In [110]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [111]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
#         print(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print_every = round(ds[0].shape[0] * 0.1)
        if i % print_every == print_every - 1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / i))
            running_loss = 0.0

print('Finished Training')

RuntimeError: size mismatch, m1: [11 x 11], m2: [54 x 120] at /Users/soumith/mc3build/conda-bld/pytorch_1549593514549/work/aten/src/TH/generic/THTensorMath.cpp:940

In [100]:
outputs = net(torch.tensor([0,0,0,0,0,0,0,0,0,0,0], dtype=torch.float32))

predicted = torch.max(outputs, 0)

predicted[1].data.cpu().numpy()

array(0)

In [101]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        input_data, labels = data
        # Note input data is batched, so output is batched
        outputs = net(input_data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (x_test.size()[0],
    100 * correct / total))

Accuracy of the network on the 20000 test images: 97 %
